# iteration1

Data:
- interaktiot kaikkien muiden ryhmistä paitsi Alman kehittäjien
- interaktiot ryhmä, ei käyttäjäkohtaisia
- kaikki yritykset - konsernien tiedot pudotetaan pois
- metadatana yrityksistä maakuntakoodi ja toimialakoodi
- Ryhmäkoko > 2 yritystä
- epochit = 5
- dataseteissä ei identiteettimatriiseja


Kysymyksiä:

1. Tuottaako malli mitenkään järkeviä suosituksia?
2. Warp vai bpr?
3. Mitä tapahtuu, jos item_featuret jättää kokonaan pois?


## Importit

In [1]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import recall_at_k
from lightfm.evaluation import reciprocal_rank
from lightfm.data import Dataset

import numpy as np
import pandas as pd

import statistics
import functools

from datetime import datetime


## Valitut metadatat yrityksille

In [9]:
SELECTED_COMPANY_FEATURES = ['location_region_code', 'industry_code']

DATA_PATH = '/mnt/d/git/masters-thesis-code/jupyter/code/data/'

## Ladataan yritysdata

In [4]:
COMPANIES_DF = pd \
        .read_csv(DATA_PATH + 'prod_data_companies_more_data_2021_09_16.csv',
                  delimiter='\t',
                  na_values='(null)',
                  dtype={
                      'business_id': str,
                      'business_id_start_date': str,
                      'company_name': str,
                      'email_address': str,
                      'industry_code': str,
                      'language': str,
                      'company_form': str,
                      'company_form_code': str,
                      'domicile_code': str,
                      'region_code': str,
                      'location_municipality_code': str,
                      'location_region_code': str,
                      'company_status_code': str,
                      'balance_industry_code': str,
                      'personnel_class': str,
                      'turnover_class': str,
                      'statement_date': str,
                      'statement_length': float,
                      'target_year': str,
                      'turnover': float,
                      'raw_materials_use': float,
                      'current_ratio': float,
                      'personnel_expenses': float,
                      'investment_expenses_cash_flow': float,
                      'ebitda': float,
                      'operating_profit_per_turnover': float,
                      'ebit': float,
                      'turnover_growth_percent': float,
                      'ebit_per_turnover': float,
                      'short_term_sales_receivable': float,
                      'net_profit': float,
                      'net_profit_per_turnover': float,
                      'aggregate_equity': float,
                      'return_on_equity_percent': float,
                      'equity_ratio': float,
                      'long_term_sales_receivable': float,
                      'quick_ratio': float,
                      'buildings': float,
                      'return_on_investment_percent': float,
                      'total_assets': float,
                      'external_services': float,
                      'aggregate_current_assets': float,
                      'personnel_average': float,
                      'concern': str,
                      'bulk': str,
                      'performer_ranking_points': float,
                      'risk_rating_class': str,
                      'latest': str
                  }
                  )


## Poistetaan konsernit (duplikaattien poistamiseksi) ja valitaan halutut sarakkeet

In [5]:
# Tästä voidaan vain poistaa konsernit, koska käytetään sellaisia metatietoja, jotka ovat samat emolla ja konsernilla

COMPANIES_DF = COMPANIES_DF[COMPANIES_DF['concern'] != 'true']
COMPANIES_DF = COMPANIES_DF[['business_id', 'company_name'] + SELECTED_COMPANY_FEATURES]
COMPANIES_DF

,business_id,company_name,location_region_code,industry_code
0,31431209,Tmi Onerva Miettunen,01,43341
1,32087307,Koti Vaihtoon LKV Oy,01,68310
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,NaN,68209
3,20469041,POLYBEAT,NaN,74909
4,24844507,SAHOJEN PUU,NaN,NaN
...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,NaN,70220
1337864,31204145,Rainmaker Voima Oy,01,78300
1337865,30137091,We Build Parks Finland Oy,19,42999
1337866,32150097,Orankien Osuuskunta,01,62010


In [44]:
COMPANIES_DF[COMPANIES_DF['business_id'].str.contains('K-')]

,business_id,company_name,location_region_code,industry_code


## Karkeutetaan toimialakoodien jakoa

Tätä voidaan vielä myöhemmin säätää, mutta lienee hyvä oletus, että alkuperäinen jako on liian tiheä

In [6]:
def transform_industry(val):
    if (pd.isnull(val)):
        return 'NaN'
    return val[0:2]

print('Alkuperäinen toimialojen määrä ', len(COMPANIES_DF['industry_code'].unique()))
COMPANIES_DF['industry_code'] = COMPANIES_DF['industry_code'].apply(transform_industry)
print('Uusi toimialojen määrä ', len(COMPANIES_DF['industry_code'].unique()))


Alkuperäinen toimialojen määrä  818
Uusi toimialojen määrä  90


## Prefiksoidaan metadatakoodit ja käsitellään puuttuvat kentät

In [7]:
def transform_str_to_keyword(val, col_name):
    if (pd.isnull(val)):
        return col_name + '+' + 'NaN'
    return col_name + '+' + val

for col_name in SELECTED_COMPANY_FEATURES:
    COMPANIES_DF[col_name] = COMPANIES_DF[col_name].apply(lambda x: transform_str_to_keyword(x, col_name))

COMPANIES_DF

,business_id,company_name,location_region_code,industry_code
0,31431209,Tmi Onerva Miettunen,location_region_code+01,industry_code+43
1,32087307,Koti Vaihtoon LKV Oy,location_region_code+01,industry_code+68
2,18601103,Teollisuuden Monialapalvelu T:mi Oksanen,location_region_code+NaN,industry_code+68
3,20469041,POLYBEAT,location_region_code+NaN,industry_code+74
4,24844507,SAHOJEN PUU,location_region_code+NaN,industry_code+NaN
...,...,...,...,...
1337863,27527742,Ingenium Partners Oy,location_region_code+NaN,industry_code+70
1337864,31204145,Rainmaker Voima Oy,location_region_code+01,industry_code+78
1337865,30137091,We Build Parks Finland Oy,location_region_code+19,industry_code+42
1337866,32150097,Orankien Osuuskunta,location_region_code+01,industry_code+62


## Käsitellään yritysdataa LightFM:n Dataset-olion luontia varten

In [8]:
ITEM_IDS = list(COMPANIES_DF['business_id'].unique())

item_features_tmp = [COMPANIES_DF[feature].unique() for feature in SELECTED_COMPANY_FEATURES]

ITEM_FEATURE_LABELS = [item for sublist in item_features_tmp for item in sublist]

ITEM_FEATURES = [(company['business_id'], 
                  [company[feature] for feature in SELECTED_COMPANY_FEATURES])
                     for company in COMPANIES_DF.to_dict(orient='records')]

print(ITEM_FEATURES[0])
print(len(ITEM_FEATURES))
print(ITEM_FEATURE_LABELS)

('31431209', ['location_region_code+01', 'industry_code+43'])
1334601
['location_region_code+01', 'location_region_code+NaN', 'location_region_code+16', 'location_region_code+02', 'location_region_code+17', 'location_region_code+11', 'location_region_code+06', 'location_region_code+07', 'location_region_code+09', 'location_region_code+12', 'location_region_code+15', 'location_region_code+14', 'location_region_code+19', 'location_region_code+13', 'location_region_code+08', 'location_region_code+05', 'location_region_code+10', 'location_region_code+18', 'location_region_code+21', 'location_region_code+04', 'industry_code+43', 'industry_code+68', 'industry_code+74', 'industry_code+NaN', 'industry_code+49', 'industry_code+01', 'industry_code+46', 'industry_code+90', 'industry_code+86', 'industry_code+56', 'industry_code+79', 'industry_code+41', 'industry_code+62', 'industry_code+87', 'industry_code+93', 'industry_code+81', 'industry_code+47', 'industry_code+22', 'industry_code+33', 'indust

## Ladataan vuorovaikutusdata

In [10]:
interactions_tmp = pd \
    .read_csv(DATA_PATH + '/interactions_2021_08_19.csv',
             delimiter='\t',
             dtype={
                 'group_id': 'string',
                 'business_id': 'string',
                 'owner': 'string'
             })

# Poistetaan vuorovaikutusdatasta sellaiset y-tunnukset, joita ei löydy kohteista
INTERACTIONS_DF = interactions_tmp[interactions_tmp.business_id.isin(ITEM_IDS)]
INTERACTIONS_DF

,group_id,business_id,owner
0,3a63222b-86b2-4293-bd2e-171011190ae6,31291154,603e1524d377150007c2dbea
1,c2626398-faac-4ff3-b02d-cdc64b50cdaa,01681709,60646431ae18cb00063ed63f
2,c2626398-faac-4ff3-b02d-cdc64b50cdaa,15055514,60646431ae18cb00063ed63f
3,c2626398-faac-4ff3-b02d-cdc64b50cdaa,01876143,60646431ae18cb00063ed63f
4,c2626398-faac-4ff3-b02d-cdc64b50cdaa,01863991,60646431ae18cb00063ed63f
...,...,...,...
548193,8b0915ff-a0cb-4520-9160-8d783a6bf308,25610905,6110c56241e21e000857ca77
548194,8b0915ff-a0cb-4520-9160-8d783a6bf308,21281841,6110c56241e21e000857ca77
548195,8b0915ff-a0cb-4520-9160-8d783a6bf308,23633484,6110c56241e21e000857ca77
548196,8b0915ff-a0cb-4520-9160-8d783a6bf308,23552270,6110c56241e21e000857ca77


## Käytetään minimiryhmäkokona 2:ta

Myöhemmin voidaan säätää, ja testailla

In [11]:
group_sizes = INTERACTIONS_DF['group_id'].value_counts()
group_sizes_df = pd.DataFrame({'group_id': group_sizes.index, 'group_size': group_sizes.values})

INTERACTIONS_WITH_GROUP_SIZES_DF = INTERACTIONS_DF.merge(group_sizes_df, on='group_id')


##### Minimiryhmäkoko = 2 #####

INTERACTIONS_2_DF = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 2]


##### Minimiryhmäkoko = 50
# Valitaan 10 minimiryhmäkooksi
#INTERACTIONS_50_DF = INTERACTIONS_WITH_GROUP_SIZES_DF[INTERACTIONS_WITH_GROUP_SIZES_DF.group_size >= 50]

INTERACTIONS_2_DF

,group_id,business_id,owner,group_size
0,3a63222b-86b2-4293-bd2e-171011190ae6,31291154,603e1524d377150007c2dbea,5
1,3a63222b-86b2-4293-bd2e-171011190ae6,08544918,603e1524d377150007c2dbea,5
2,3a63222b-86b2-4293-bd2e-171011190ae6,10134299,603e1524d377150007c2dbea,5
3,3a63222b-86b2-4293-bd2e-171011190ae6,26404708,603e1524d377150007c2dbea,5
4,3a63222b-86b2-4293-bd2e-171011190ae6,29747968,603e1524d377150007c2dbea,5
...,...,...,...,...
525508,8b0915ff-a0cb-4520-9160-8d783a6bf308,25610905,6110c56241e21e000857ca77,131
525509,8b0915ff-a0cb-4520-9160-8d783a6bf308,21281841,6110c56241e21e000857ca77,131
525510,8b0915ff-a0cb-4520-9160-8d783a6bf308,23633484,6110c56241e21e000857ca77,131
525511,8b0915ff-a0cb-4520-9160-8d783a6bf308,23552270,6110c56241e21e000857ca77,131


In [12]:
def print_interactions_meta_data(interactions_df):
    print('ryhmiä: {groups}, interaktioita {interactions}, yrityksiä {companies}'
          .format(groups=len(list(interactions_df['group_id'].unique())),
                  interactions=interactions_df.shape[0], 
                  companies=len(list(interactions_df['business_id'].unique()))))

print('----- group_size>=1 -----')
print_interactions_meta_data(INTERACTIONS_DF)

print('\n----- group_size>=2 -----')
print_interactions_meta_data(INTERACTIONS_2_DF)

#print('\n----- group_size>=50 -----')
#print_interactions_meta_data(INTERACTIONS_50_DF)

----- group_size>=1 -----
ryhmiä: 1399, interaktioita 525513, yrityksiä 140677

----- group_size>=2 -----
ryhmiä: 1305, interaktioita 525419, yrityksiä 140676


## Luodaan LightFM:n ymmärtämä Dataset-olio

In [13]:
def create_dataset(interactions_df):
    dataset = Dataset(user_identity_features=False, item_identity_features=False) # Näiden togglaus-testi myöhemmässä iteraatiossa
    
    interactions = [(interaction['group_id'], interaction['business_id']) 
                for interaction in interactions_df.to_dict(orient='records')]
    
    user_ids = list(set(interactions_df['group_id'].values))

    dataset.fit(users=user_ids, items=ITEM_IDS, item_features=ITEM_FEATURE_LABELS)
    
    (interactions_ds, weights_ds) = dataset.build_interactions(interactions)
    
    item_features_ds = dataset.build_item_features(ITEM_FEATURES, normalize=False)
    
    return (interactions_ds, item_features_ds, dataset.mapping())    

In [21]:
#(INTERACTIONS_DS, ITEM_FEATURES_DS) = create_dataset(INTERACTIONS_DF)
#print(repr(INTERACTIONS_DS))

In [14]:
(INTERACTIONS_2_DS, ITEM_FEATURES_2_DS, DATASET_MAPPING_2) = create_dataset(INTERACTIONS_2_DF)
USER_MAP_2 = DATASET_MAPPING_2[0]
ITEM_MAP_2 = DATASET_MAPPING_2[2]
ITEM_FEATURE_MAP_2 = DATASET_MAPPING_2[3]
print(repr(INTERACTIONS_2_DS))

<1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
	with 525419 stored elements in COOrdinate format>


In [15]:
#(INTERACTIONS_50_DS, ITEM_FEATURES_50_DS) = create_dataset(INTERACTIONS_50_DF)
#print(repr(INTERACTIONS_50_DS))

## Luodaan cross-validationia varten ositetut datasetit

In [16]:
def create_partitioned_datasets(interactions_ds):
    (rest, fifth_1) = random_train_test_split(interactions_ds, test_percentage=0.2)
    (rest, fifth_2) = random_train_test_split(rest, test_percentage=0.25)
    (rest, fifth_3) = random_train_test_split(rest, test_percentage=0.3333333)
    (fifth_4, fifth_5) = random_train_test_split(rest, test_percentage=0.5)
    
    return [fifth_1, fifth_2, fifth_3, fifth_4, fifth_5]
    

In [17]:
#INTERACTIONS_CV = create_partitioned_datasets(INTERACTIONS_DS)
INTERACTIONS_2_CV = create_partitioned_datasets(INTERACTIONS_2_DS)
#INTERACTIONS_50_CV = create_partitioned_datasets(INTERACTIONS_50_DS)
INTERACTIONS_2_CV

[<1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
 	with 105084 stored elements in COOrdinate format>,
 <1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
 	with 105084 stored elements in COOrdinate format>,
 <1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
 	with 105084 stored elements in COOrdinate format>,
 <1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
 	with 105083 stored elements in COOrdinate format>,
 <1305x1334601 sparse matrix of type '<class 'numpy.int32'>'
 	with 105084 stored elements in COOrdinate format>]

## Arvioidaan mallien laatua

In [18]:
def run_evaluation_function(model, test_ds, train_ds, evaluation_function, name, item_features=None):    
    print('Calculating {name} for train dataset...'.format(name=name))
    train_results = evaluation_function(model, train_ds, item_features=item_features, num_threads=10)
    np.savetxt('iteration1-train-results-{}.txt'.format(name), train_results)
    train_metric = train_results.mean()
    
    print('Calculating {name} for test dataset...'.format(name=name))
    test_results = evaluation_function(model, test_ds, item_features=item_features, num_threads=10)
    np.savetxt('iteration1-test-results-{}.txt'.format(name), test_results)
    test_metric = test_results.mean()
    
    print('{name}: train {train_metric:.2f}, test {test_metric:.2f}'.format(name=name, 
                                                                            train_metric=train_metric, 
                                                                            test_metric=test_metric))
    print('\n')
    return (train_metric, test_metric)

def run_evaluations_for_ds(model, train_ds, test_ds, model_name, item_features=None):
    auc = run_evaluation_function(model, test_ds, train_ds, auc_score, 'AUC_' + model_name, item_features)
    precision = run_evaluation_function(model, test_ds, train_ds, precision_at_k, 'PRECISION_' + model_name, item_features)
    recall = run_evaluation_function(model, test_ds, train_ds, recall_at_k, 'RECALL_' + model_name, item_features)
    reciprocal = run_evaluation_function(model, test_ds, train_ds, reciprocal_rank, 'RECIPROCAL_' + model_name, item_features)
    
    return (auc, precision, recall, reciprocal)
    

def run_evaluations(interactions_cv, item_features_ds, model_epochs):
    
    results = {
        'WARP': [],
        'BPR': [],
        'WARP_NO_ITEM': [],
        'BPR_NO_ITEM': []
    }
    
    for i in range(0, len(interactions_cv)):
        print('Starting iteration {}...'.format(i))    
            
        test_ds = interactions_cv[i]
        
        # laitetaan uuteen listaan kaikki paitsi testidatasetti
        train_ds_tmp = [ds for j,ds in enumerate(interactions_cv) if j != i]
        # yhdistetään treenidatasetiksi valikoituneet vuorovaikutusmatriisit
        train_ds = functools.reduce(lambda a,b: a + b, train_ds_tmp)
        
        MODEL_WARP = LightFM(loss='warp')
        MODEL_WARP.fit(train_ds, item_features=item_features_ds, epochs=model_epochs, num_threads=10, verbose=True)

        MODEL_BPR = LightFM(loss='bpr')
        MODEL_BPR.fit(train_ds, item_features=item_features_ds, epochs=model_epochs, num_threads=10, verbose=True)

        MODEL_WARP_NO_ITEM = LightFM(loss='warp')
        MODEL_WARP_NO_ITEM.fit(train_ds, epochs=model_epochs, num_threads=10, verbose=True)

        MODEL_BPR_NO_ITEM = LightFM(loss='bpr')
        MODEL_BPR_NO_ITEM.fit(train_ds, epochs=model_epochs, num_threads=10, verbose=True)
        
        results['WARP'].append(run_evaluations_for_ds(MODEL_WARP, train_ds, test_ds, 'WARP_RUN_{}'.format(i), item_features_ds))
        results['BPR'].append(run_evaluations_for_ds(MODEL_BPR, train_ds, test_ds, 'BPR_RUN_{}'.format(i), item_features_ds))
        results['WARP_NO_ITEM'].append(run_evaluations_for_ds(MODEL_WARP_NO_ITEM, train_ds, test_ds, 'WARP_NO_ITEM_RUN_{}'.format(i)))
        results['BPR_NO_ITEM'].append(run_evaluations_for_ds(MODEL_BPR_NO_ITEM, train_ds, test_ds, 'BPR_NO_ITEM_RUN_{}'.format(i)))
        
    return results


In [19]:
def print_simple_results(results_tuple):
    auc, precision, recall, reciprocal = results_tuple
    print('AUC: \n train {} \n test {}'.format(auc[0], auc[1]))
    print('PRECISION: \n train {} \n test {}'.format(precision[0], precision[1]))
    print('RECALL: \n train {} \n test {}'.format(recall[0], recall[1]))
    print('RECIPROCAL: \n train {} \n test {}'.format(reciprocal[0], reciprocal[1]))    
    

def print_metric_result(result_arr, model_name):
    train_results = [x[0] for x in result_arr]
    test_results = [x[1] for x in result_arr]
    
    print('{name}:\n train mean {train_mean:.2f} ({train_arr})\n test mean {test_mean:.2f} ({test_arr})\n'
          .format(train_mean=statistics.mean(train_results),
                 test_mean=statistics.mean(test_results),
                 train_arr=['%.2f' % x for x in train_results],
                 test_arr=['%.2f' % x for x in test_results],
                 name=model_name))
    

def print_all_results(results):
    for i,metric in enumerate(['AUC', 'PRECISION', 'RECALL', 'RECIPROCAL']):
        print('\n-----{}-----'.format(metric))
        for model_name,result_arr in results.items():
            print_metric_result([res[i] for res in result_arr], model_name)
    

### group_size >= 2

In [21]:
print(datetime.now())
RESULTS_2 = run_evaluations(INTERACTIONS_2_CV, ITEM_FEATURES_2_DS, 5)
print(datetime.now())

2022-04-03 20:55:18.729765
Starting iteration 0...


Epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.83it/s]


Calculating AUC_WARP_RUN_0 for train dataset...
Calculating AUC_WARP_RUN_0 for test dataset...
AUC_WARP_RUN_0: train 0.97, test 0.91


Calculating PRECISION_WARP_RUN_0 for train dataset...
Calculating PRECISION_WARP_RUN_0 for test dataset...
PRECISION_WARP_RUN_0: train 0.00, test 0.00


Calculating RECALL_WARP_RUN_0 for train dataset...


/home/sakaris/miniconda3/lib/python3.9/site-packages/lightfm/evaluation.py:166: RuntimeWarning: invalid value encountered in true_divide
  return hit / retrieved


Calculating RECALL_WARP_RUN_0 for test dataset...
RECALL_WARP_RUN_0: train nan, test nan


Calculating RECIPROCAL_WARP_RUN_0 for train dataset...
Calculating RECIPROCAL_WARP_RUN_0 for test dataset...
RECIPROCAL_WARP_RUN_0: train 0.01, test 0.00


Calculating AUC_BPR_RUN_0 for train dataset...
Calculating AUC_BPR_RUN_0 for test dataset...
AUC_BPR_RUN_0: train 0.94, test 0.88


Calculating PRECISION_BPR_RUN_0 for train dataset...
Calculating PRECISION_BPR_RUN_0 for test dataset...
PRECISION_BPR_RUN_0: train 0.00, test 0.00


Calculating RECALL_BPR_RUN_0 for train dataset...
Calculating RECALL_BPR_RUN_0 for test dataset...
RECALL_BPR_RUN_0: train nan, test nan


Calculating RECIPROCAL_BPR_RUN_0 for train dataset...
Calculating RECIPROCAL_BPR_RUN_0 for test dataset...
RECIPROCAL_BPR_RUN_0: train 0.01, test 0.00


Calculating AUC_WARP_NO_ITEM_RUN_0 for train dataset...
Calculating AUC_WARP_NO_ITEM_RUN_0 for test dataset...
AUC_WARP_NO_ITEM_RUN_0: train 0.99, test 0.90


Calculating PRECISIO

Epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.03it/s]


Calculating AUC_WARP_RUN_1 for train dataset...
Calculating AUC_WARP_RUN_1 for test dataset...
AUC_WARP_RUN_1: train 0.97, test 0.92


Calculating PRECISION_WARP_RUN_1 for train dataset...
Calculating PRECISION_WARP_RUN_1 for test dataset...
PRECISION_WARP_RUN_1: train 0.01, test 0.00


Calculating RECALL_WARP_RUN_1 for train dataset...
Calculating RECALL_WARP_RUN_1 for test dataset...
RECALL_WARP_RUN_1: train nan, test nan


Calculating RECIPROCAL_WARP_RUN_1 for train dataset...
Calculating RECIPROCAL_WARP_RUN_1 for test dataset...
RECIPROCAL_WARP_RUN_1: train 0.01, test 0.01


Calculating AUC_BPR_RUN_1 for train dataset...
Calculating AUC_BPR_RUN_1 for test dataset...
AUC_BPR_RUN_1: train 0.94, test 0.89


Calculating PRECISION_BPR_RUN_1 for train dataset...
Calculating PRECISION_BPR_RUN_1 for test dataset...
PRECISION_BPR_RUN_1: train 0.00, test 0.00


Calculating RECALL_BPR_RUN_1 for train dataset...
Calculating RECALL_BPR_RUN_1 for test dataset...
RECALL_BPR_RUN_1: train nan, test

Epoch: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  7.06it/s]


Calculating AUC_WARP_RUN_2 for train dataset...
Calculating AUC_WARP_RUN_2 for test dataset...
AUC_WARP_RUN_2: train 0.97, test 0.92


Calculating PRECISION_WARP_RUN_2 for train dataset...
Calculating PRECISION_WARP_RUN_2 for test dataset...
PRECISION_WARP_RUN_2: train 0.00, test 0.00


Calculating RECALL_WARP_RUN_2 for train dataset...
Calculating RECALL_WARP_RUN_2 for test dataset...
RECALL_WARP_RUN_2: train nan, test nan


Calculating RECIPROCAL_WARP_RUN_2 for train dataset...
Calculating RECIPROCAL_WARP_RUN_2 for test dataset...
RECIPROCAL_WARP_RUN_2: train 0.01, test 0.01


Calculating AUC_BPR_RUN_2 for train dataset...
Calculating AUC_BPR_RUN_2 for test dataset...
AUC_BPR_RUN_2: train 0.94, test 0.88


Calculating PRECISION_BPR_RUN_2 for train dataset...
Calculating PRECISION_BPR_RUN_2 for test dataset...
PRECISION_BPR_RUN_2: train 0.00, test 0.00


Calculating RECALL_BPR_RUN_2 for train dataset...
Calculating RECALL_BPR_RUN_2 for test dataset...
RECALL_BPR_RUN_2: train nan, test

In [36]:
print_all_results(RESULTS_2)


-----AUC-----
WARP:
 train mean 0.97 (['0.97', '0.97', '0.97', '0.97', '0.97'])
 test mean 0.97 (['0.97', '0.97', '0.97', '0.97', '0.97'])

BPR:
 train mean 0.94 (['0.94', '0.94', '0.94', '0.94', '0.94'])
 test mean 0.93 (['0.93', '0.94', '0.93', '0.93', '0.93'])

WARP_NO_ITEM:
 train mean 0.99 (['0.99', '0.99', '0.99', '0.99', '0.99'])
 test mean 0.95 (['0.95', '0.95', '0.95', '0.94', '0.95'])

BPR_NO_ITEM:
 train mean 0.87 (['0.86', '0.87', '0.87', '0.87', '0.87'])
 test mean 0.83 (['0.83', '0.83', '0.84', '0.82', '0.83'])


-----PRECISION-----
WARP:
 train mean 0.03 (['0.03', '0.02', '0.03', '0.03', '0.03'])
 test mean 0.01 (['0.01', '0.01', '0.01', '0.01', '0.01'])

BPR:
 train mean 0.01 (['0.01', '0.02', '0.01', '0.01', '0.02'])
 test mean 0.00 (['0.00', '0.00', '0.00', '0.00', '0.00'])

WARP_NO_ITEM:
 train mean 0.18 (['0.18', '0.17', '0.19', '0.18', '0.19'])
 test mean 0.03 (['0.03', '0.03', '0.04', '0.04', '0.03'])

BPR_NO_ITEM:
 train mean 0.19 (['0.18', '0.19', '0.18', '0.20